In [ ]:
!pip install -qU "semantic-router[pinecone]>=0.1.5"

In [2]:
from semantic_router import Route

# we could use this as a guide for our chatbot to avoid political conversations
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the presidentdon't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

# this could be used as an indicator to our chatbot to switch to a more
# conversational prompt
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

# we place both of our decisions together into single list
routes = [politics, chitchat]

In [3]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

# get at platform.openai.com
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API key: "
)
encoder = OpenAIEncoder()

In [4]:
from semantic_router.index.pinecone import PineconeIndex

# get at app.pinecone.io
os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter Pinecone API key: "
)
pc_index = PineconeIndex()

2025-01-07 12:10:29 - pinecone_plugin_interface.logging - INFO - discover_namespace_packages.py:12 - discover_subpackages() - Discovering subpackages in _NamespacePath(['c:\\Users\\Joshu\\OneDrive\\Documents\\Aurelio\\agents-course\\07-pratical-ai\\.venv\\Lib\\site-packages\\pinecone_plugins'])
2025-01-07 12:10:29 - pinecone_plugin_interface.logging - INFO - discover_plugins.py:9 - discover_plugins() - Looking for plugins in pinecone_plugins.inference
2025-01-07 12:10:29 - pinecone_plugin_interface.logging - INFO - installation.py:10 - install_plugins() - Installing plugin inference into Pinecone
2025-01-07 12:10:31 - pinecone_plugin_interface.logging - INFO - discover_namespace_packages.py:12 - discover_subpackages() - Discovering subpackages in _NamespacePath(['c:\\Users\\Joshu\\OneDrive\\Documents\\Aurelio\\agents-course\\07-pratical-ai\\.venv\\Lib\\site-packages\\pinecone_plugins'])
2025-01-07 12:10:31 - pinecone_plugin_interface.logging - INFO - discover_plugins.py:9 - discover_pl

In [5]:
from semantic_router.routers import SemanticRouter

rl = SemanticRouter(encoder=encoder, routes=routes, index=pc_index, auto_sync="local")

2025-01-07 12:11:05 - pinecone_plugin_interface.logging - INFO - discover_namespace_packages.py:12 - discover_subpackages() - Discovering subpackages in _NamespacePath(['c:\\Users\\Joshu\\OneDrive\\Documents\\Aurelio\\agents-course\\07-pratical-ai\\.venv\\Lib\\site-packages\\pinecone_plugins'])
2025-01-07 12:11:05 - pinecone_plugin_interface.logging - INFO - discover_plugins.py:9 - discover_plugins() - Looking for plugins in pinecone_plugins.inference
2025-01-07 12:11:08 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


We can check our route layer and index information.

In [6]:
rl.list_route_names()

['politics', 'chitchat']

In [7]:
len(rl.index)

10

We can also view all of the records for a given route:

In [8]:
rl.index._get_route_ids(route_name="politics")

['politics#64069085d9d6e98e5a80915f69fabe82bac6c742f801bc305c5001dce88f0d19',
 'politics#af8b76111f260cf44fb34f04fcf82927dcbe08e8f47c30f4d571379c1512fac8',
 'politics#d1bb40236c3d95b9c695bfa86b314b6da4eb87e136699563fccae47fccea23e2',
 'politics#ed0f3dd7bd5dea12e55b1953bcd2c562a5ab19f501f6d5ff8c8927652c3904b8',
 'politics#fc6d15f9e6075e6de82b3fbef6722b64353e4eadc8d663b7312a4ed60c43e6f6']

And query:

In [9]:
rl("don't you love politics?").name

2025-01-07 12:11:37 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


'politics'

In [10]:
rl("how's the weather today?").name

2025-01-07 12:11:43 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


'chitchat'

In [11]:
rl("I'm interested in learning about llama 2").name

2025-01-07 12:11:46 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


We can delete or update routes.

In [12]:
len(rl.index)

10

In [13]:
import time

rl.delete(route_name="chitchat")
time.sleep(1)
len(rl.index)

2025-01-07 12:11:57 - semantic_router.utils.logger - WARNING - pinecone.py:431 - _read_config() - Configuration for sr_lock parameter not found in index.


5

In [14]:
rl("how's the weather today?").name

2025-01-07 12:12:01 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [15]:
rl.index.get_routes()

[Route(name='politics', utterances=["they're going to destroy this country!", 'they will save the country!', "don't you just love the presidentdon't you just hate the president", "isn't politics the best thing ever", "why don't you tell me about your political opinions"], description=None, function_schemas=None, llm=None, score_threshold=None, metadata={})]

In [16]:
rl.index.describe()

{'type': 'pinecone', 'dimensions': 1536, 'vectors': 5}